### First attempts at training


##### some resources I've been using
huggingface decision transformer:https://huggingface.co/docs/transformers/en/model_doc/decision_transformer

training huggingface decision transformer: https://github.com/huggingface/blog/blob/main/notebooks/101_train-decision-transformers.ipynb


custom gym environments:https://www.gymlibrary.dev/content/environment_creation/

gymnasium documentation: https://gymnasium.farama.org/

decision transformer github: https://github.com/kzl/decision-transformer/tree/master

very simple DT: https://github.com/nikhilbarhate99/min-decision-transformer





In [3]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from datasets import load_dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments

from TensorRTS import TensorRTS

from entity_gym.env import Observation
from entity_gym.env import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# import tensorRTS

In [61]:
#gymnasium wrapper for tensorRTS; I did not finish this but it may be useful in the future

# import gymnasium as gym
# import numpy as np
# from gymnasium import spaces


# class CustomEnv(gym.Env):
#     """Custom Environment that follows gym interface."""

#     metadata = {"render_modes": ["human"], "render_fps": 30}

#     def __init__(self):
#         super().__init__()


#         self.tensor = TensorRTS();
#         # Define action and observation space
#         # They must be gym.spaces objects
#         # Example when using discrete actions:
#         self.action_space = spaces.utils.flatten_space(spaces.Discrete(4))
#         # Example for using image as input (channel-first; channel-last also works):
#         # self.observation_space = spaces.Box(low=0, high=255,
#         #                                     shape=(N_CHANNELS, HEIGHT, WIDTH), dtype=np.uint8)
    
        

#         self.observation_space_unflatten = spaces.Dict({
#              "Cluster": spaces.Box(low=0, high=self.tensor.maxdots, shape=(self.tensor.nclusters, 2),dtype=int),
#              "Tensor": spaces.Box(low=0, high=100, shape=(2, 4), dtype=int)
#             }
#             )
#         self.observation_space = spaces.utils.flatten_space(self.observation_space_unflatten)

#     def step(self, action):
#         ...
#         return observation, reward, terminated, truncated, info

#     def reset(self, seed=None, options=None):
#         super().reset()
#         self.tensor.reset()
#         return self._get_obs()

#     def render(self):
#         ...

#     def close(self):
#         ...


#     def _get_obs(self):
#             return {"Cluster": self.tensor.clusters, "Tensor": self.tensor.tensors}

In [4]:
#thank you ai
import functools
def flatten_dict_of_arrays(data, prefix=""):
  flattened_list = []
  def flatten_inner(value, key):
    if isinstance(value, list):
      for i, v in enumerate(value):
        flatten_inner(v, f"{prefix}{key}[{i}]")
    else:
      flattened_list.append((f"{prefix}{key}", value))

  for key, value in data.items():
    flatten_inner(value, key)

  return [value for _, value in flattened_list]


def value_to_discrete_4(input):
    output = [0,0,0,0]
    output[input] = 1
    return output

### Generate dataset

In [5]:
#This generates game state data using a random agent vs rush agent
from entity_gym.env import * #for GlobalCategoricalAction
possible_moves = {
    0 : "advance",
    1 : "retreat",
    2 : "rush",
    3 : "boom"
}
rdmEnv = TensorRTS(enable_prinouts = False)
rdmEnv.reset()
rndDataset = {
    'observations': [],
    'actions': [], 
    'rewards': [], 
    'dones': [],
    
}
action = {}
for _ in range(100):
    rdmEnv.reset()

    o = []
    a = []
    r = []
    d = []
    
    for x in range(100):
        o.append(flatten_dict_of_arrays(rdmEnv.observe().features))
        move = random.randrange(0, 4)
        action["Move"] = GlobalCategoricalAction(0, possible_moves[move])
        a.append(value_to_discrete_4(move))

        model_out = rdmEnv.act(action)

        r.append(model_out.reward)

        d.append(model_out.done)
        if model_out.done == True: 
            break 

    rndDataset['observations'].append(o)
    rndDataset['actions'].append(a)
    rndDataset['rewards'].append(r)
    rndDataset['dones'].append(d)
    # rndList.append(rndDataset)

from datasets import Dataset
ds = Dataset.from_dict(rndDataset)


### Load in base model for training and collator

In [6]:
import DTscripts

collator = DTscripts.DecisionTransformerGymDataCollator(ds)
config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim)
model = DTscripts.TrainableDT(config)

### Training

In [7]:

# model = DTscripts.TrainableDT.from_pretrained("./output/models4")
import wandb
# wandb.init(project="VectorRTS-transformer")

os.environ["WANDB_DISABLED"] = "true" 
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=20,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
    use_mps_device=True,
    # evaluation_strategy="steps",
    # eval_steps = 10

)
training_args = training_args.set_save(strategy="steps", steps=10)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    data_collator=collator,
)

trainer.train()
trainer.save_model("./output/models20")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dgovorov7. Use `wandb login --relogin` to force relogin


  0%|          | 0/40 [00:00<?, ?it/s]

{'train_runtime': 9.2059, 'train_samples_per_second': 217.251, 'train_steps_per_second': 4.345, 'train_loss': 0.21411147117614746, 'epoch': 20.0}


In [ ]:
#stolen
# def get_action(model, states, actions, rewards, returns_to_go, timesteps):
#     # This implementation does not condition on past rewards

#     states = states.reshape(1, -1, model.config.state_dim)
#     actions = actions.reshape(1, -1, model.config.act_dim)
#     returns_to_go = returns_to_go.reshape(1, -1, 1)
#     timesteps = timesteps.reshape(1, -1)

#     states = states[:, -model.config.max_length :]
#     actions = actions[:, -model.config.max_length :]
#     returns_to_go = returns_to_go[:, -model.config.max_length :]
#     timesteps = timesteps[:, -model.config.max_length :]
#     padding = model.config.max_length - states.shape[1]
#     # pad all tokens to sequence length
#     attention_mask = torch.cat([torch.zeros(padding), torch.ones(states.shape[1])])
#     attention_mask = attention_mask.to(dtype=torch.long).reshape(1, -1)
#     states = torch.cat([torch.zeros((1, padding, model.config.state_dim)), states], dim=1).float()
#     actions = torch.cat([torch.zeros((1, padding, model.config.act_dim)), actions], dim=1).float()
#     returns_to_go = torch.cat([torch.zeros((1, padding, 1)), returns_to_go], dim=1).float()
#     timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long), timesteps], dim=1)

#     state_preds, action_preds, return_preds = model.original_forward(
#         states=states,
#         actions=actions,
#         rewards=rewards,
#         returns_to_go=returns_to_go,
#         timesteps=timesteps,
#         attention_mask=attention_mask,
#         return_dict=False,
#     )

#     return action_preds[0, -1]

### Evaluation

In [8]:

def get_move_no_lookback(state):
    state_dim = 20
    act_dim = 4
    states = torch.from_numpy(state).reshape(1, 1, state_dim).to(device=device, dtype=torch.float32)
    actions = torch.zeros((1, 1, act_dim), device=device, dtype=torch.float32)
    rewards = torch.zeros(1, 1, device=device, dtype=torch.float32)
    target_return = torch.tensor(10, dtype=torch.float32).reshape(1, 1)
    timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
    attention_mask = torch.zeros(1, 1, device=device, dtype=torch.float32)


    with torch.no_grad():
        state_preds, action_preds, return_preds = model(
            states=states,
            actions=actions,
            rewards=rewards,
            returns_to_go=target_return,
            timesteps=timesteps,
            attention_mask=attention_mask,
            return_dict=False,
            # use_mps_device=True

        )
    return state_preds, action_preds, return_preds

def evaluate_model(model,num_runs):
    testEnv = TensorRTS(enable_prinouts = False)
    possible_moves = {
        0 : "advance",
        1 : "retreat",
        2 : "rush",
        3 : "boom"
    }
    wins = 0
    losses = 0
    ties = 0
    for _ in range(num_runs):

        testEnv.reset()
        for _ in range(1000):
            #make prediction
            model_out = get_move_no_lookback(np.array(flatten_dict_of_arrays(testEnv.observe().features)))
            action['Move'] = GlobalCategoricalAction(0, possible_moves[torch.argmax(model_out[1]).item()])
            pred_reward = model_out[2]
            result = testEnv.act(action)
            # print(model_out[1])

            if result.done == True: 
                break 
            

        # print(result.reward)
        if (result.reward == 10): 
            wins+=1
            print(wins)
        if (result.reward == 0): ties +=1
        if (result.reward == -10): losses+=1


    return ((wins+ties)/num_runs)
model = DecisionTransformerModel.from_pretrained("./output/models4")

evaluate_model(model,100) #Returns win rate + tie rate in %

0.26

In [2]:
# model = DecisionTransformerModel.from_pretrained("./output/models3")
# # evaluation
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# possible_moves = {
#     0 : "advance",
#     1 : "retreat",
#     2 : "rush",
#     3 : "boom"
# }
# action = {}


# testEnv = TensorRTS(enable_prinouts = False)

# total_reward = 0

# for _ in range(10):

#     testEnv.reset()
#     for _ in range(100):
#         #make prediction
#         model_out = get_move_no_lookback(np.array(flatten_dict_of_arrays(testEnv.observe().features)))
#         action['Move'] = GlobalCategoricalAction(0, possible_moves[torch.argmax(model_out[1]).item()])
#         pred_reward = model_out[2]
#         result = testEnv.act(action)
#         # print(model_out[1])

#         if result.done == True: 
#             break 

#     total_reward += result.reward

    



# total_reward

NameError: name 'get_move_no_lookback' is not defined